In [1]:
# Define the gpu  on the gpu machine
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

import pandas as pd
import json
from tqdm import tqdm
from transformers import AutoTokenizer
from datasets import load_dataset
import numpy as np
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
import re

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


/home/tajak/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Tokenize and count tokens for train_df

Code for tokenization (it is now already done):

In [18]:
# Import the train dataset
train = load_dataset("TajaKuzman/X-GENRE-multilingual-text-genre-dataset", "train")

# To open as Pandas DataFrame:
train_df = pd.DataFrame(train["train"])

display(train_df.head(2))

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# Tokenize the train dataset
tokens_train = []
integers_train = []
token_list_train = []

for text in tqdm(train_df.text.to_list()):
	encoded_text = tokenizer(text)
	# Take all tokens_train, except the beginning (<s>) and end (</s>) token
	current_tokens_train = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)[1:-1]
	tokens_train.append(current_tokens_train)
	token_list_train.extend(current_tokens_train)
	integers_train.append(encoded_text.input_ids[1:-1])

train_df["tokens_train"] = tokens_train
train_df["token_ids"] = integers_train

#print(token_list_train[:10])
#print(len(token_list_train))

# Create a list of tokens, where we take only the first 512 tokens
train_tokens_shortened = []

for i in train_df["tokens_train"].to_list():
	train_tokens_shortened.extend(i[:512])

print(len(train_tokens_shortened))

train_df.head(3)

# Save the tokenized version
train_df.to_json("datasets/tokenized_datasets/X-GENRE-train-tokenized.json")

,text,labels,dataset,language
0,"Seeking All Things Brilliant ""I want people to...",Other,CORE,English
1,Meet Orchid du Bois I first met Hayley Mowday ...,Other,CORE,English


100%|██████████| 1772/1772 [00:09<00:00, 189.74it/s]


699465


Code to count tokens:

In [2]:
# Open the tokenized df
train_df = pd.read_json("datasets/tokenized_datasets/X-GENRE-train-tokenized.json")
train_df.head(2)

,text,labels,dataset,language,tokens_train,token_ids,text_norm,tokens_train_norm
0,"Seeking All Things Brilliant ""I want people to...",Other,CORE,English,"[▁See, king, ▁All, ▁Things, ▁Br, illian, t, ▁""...","[6872, 6048, 3164, 119175, 13008, 162076, 18, ...","seeking all things brilliant ""i want people to...","[▁seeking, ▁all, ▁things, ▁brilliant, ▁"", i, ▁..."
1,Meet Orchid du Bois I first met Hayley Mowday ...,Other,CORE,English,"[▁Meet, ▁Or, ch, id, ▁du, ▁Bo, is, ▁I, ▁first,...","[72626, 3347, 206, 532, 115, 2460, 164, 87, 51...",meet orchid du bois i first met hayley mowday ...,"[▁meet, ▁or, ch, id, ▁du, ▁bois, ▁i, ▁first, ▁..."


In [3]:
# Create a dictionary that counts all the token occurrences

# Create a list of tokens, where we take only the first 512 tokens
train_tokens_shortened = []

for i in train_df["tokens_train"].to_list():
	train_tokens_shortened.extend(i[:512])

print(len(train_tokens_shortened))

# Create a dictionary which counts the occurrences of the words

word_dict_train = Counter(train_tokens_shortened)

# Sort the dictionary alphabetically based on keys
word_dict_train = dict(sorted(word_dict_train.items()))

print(list(word_dict_train.items())[:100])
print(len(word_dict_train))

699465
[('!', 430), ('!!', 23), ('!!!', 14), ('!!!!', 6), ('!!!!!', 1), ('!!!!!!', 1), ('!!!!!!!', 1), ('!"', 14), ('!)', 10), ('!),', 1), ('!).', 2), ('"', 528), ('")', 7), ('"),', 6), ('").', 7), ('",', 56), ('".', 83), ('"...', 3), ('";', 3), ('"?', 6), ('#', 4), ('$', 8), ('%', 4), ('%)', 1), ('&', 46), ("'", 4517), ('(', 31), ('(1', 1), (')', 788), ('),', 242), (').', 297), ('):', 13), (');', 5), ('*', 19), ('**', 2), ('****', 1), ('+', 15), ('+5', 1), (',', 23447), (',«', 12), ('-', 2803), ('---', 3), ('------', 5), ('----------------', 41), ('-0', 3), ('-01', 1), ('-02', 2), ('-02-', 2), ('-03-', 4), ('-06', 1), ('-06-', 3), ('-09-', 1), ('-1', 15), ('-1)', 3), ('-10', 2), ('-10-', 1), ('-11', 6), ('-11-', 1), ('-12', 8), ('-13', 6), ('-14', 3), ('-15', 7), ('-16', 9), ('-17', 3), ('-18', 9), ('-19', 5), ('-2', 10), ('-20', 3), ('-2000', 3), ('-2005', 1), ('-2007', 8), ('-2009', 1), ('-2010', 1), ('-2011', 1), ('-2012', 1), ('-2014', 1), ('-2020', 1), ('-21', 4), ('-22', 3), ('-

In [5]:
len(set(train_tokens_shortened))

27025

The train dataset has 699.465 tokens and 27.025 unique words.

In [4]:
# See the most frequent tokens:

# Sort the dictionary by values (word counts) in descending order
sorted(word_dict_train.items(), key=lambda x: x[1], reverse=True)[:10]


[(',', 23447),
 ('.', 21407),
 ('▁', 19553),
 ('▁the', 18860),
 ('s', 14184),
 ('▁to', 10762),
 ('▁of', 9912),
 ('▁and', 9752),
 ('▁in', 9140),
 ('▁a', 8341)]

In [14]:
# Save the dictionary of tokens
with open("datasets/tokenized_datasets/X-GENRE-train-token-count.json", "w") as train_count_file:
	json.dump(word_dict_train, train_count_file)

# Tokenize and count tokens for test sets & calculate percentage overlap

Code with which I tokenized the datasets:

In [5]:
# Import the final dataset with test sets
with open("manual-annotations/multilingual-genre-annotated-test-set.json") as main_file:
	main_dict = json.load(main_file)

main_dict.keys()

dict_keys(['mt', 'el', 'tr', 'sq', 'is', 'uk', 'ca', 'mk', 'hr', 'sl'])

In [7]:
# Code with which the test sets were tokenized
# Improved code to calculate token overlap - we can simply calculate which tokens do not overlap - meaning that they occur only in one of the dataset, and calculate the percentage overlap based on that

# Define an array of token overlap
token_overlap_results = {}

# Loop through the datasets and calculate token overlap
for lang in list(main_dict.keys()):
	df = pd.DataFrame(main_dict[lang]["dataset"])

	tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

	tokens = []
	integers = []
	token_list = []

	print("Tokenizing text.")

	for text in tqdm(df.text.to_list()):
		encoded_text = tokenizer(text)
		# Take all tokens, except the beginning (<s>) and end (</s>) token
		current_tokens = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)[1:-1]
		# Shorten the list to 512, as tokens after that were not observed by the classifier
		current_tokens = current_tokens[:512]
		tokens.append(current_tokens)
		token_list.extend(current_tokens)
		integers.append(encoded_text.input_ids[1:-1][:512])

	df["tokens"] = tokens
	df["token_ids"] = integers

	print(token_list[:10])
	print("All tokens:")
	print(len(token_list))

	token_overlap_results = {}

	# See how many tokens do not overlap
	no_overlap_counter = 0

	print("Calculating overlap.")

	# Save tokens that overlap for further inspection
	overlap_token_list = []

	# We calculate overlap by counting how many tokens do not appear in both sets
	for token in tqdm(token_list):
		if token not in train_tokens_shortened:
			no_overlap_counter += 1
		else:
			overlap_token_list.append(token)

	# Out of all tokens in test set, how many do not overlap with train set?
	no_overlap_per = no_overlap_counter/len(token_list)

	# Calculate percentage of overlap based on that
	overlap_per = 1-no_overlap_per

	print(f"Number of tokens that overlap: {len(overlap_token_list)}")
	print(f"Number of different tokens that overlap: {len(set(overlap_token_list))}")
	print(f"Percentage of overlap: {overlap_per}")

	# Update the dataset in the dictionary
	main_dict[lang]["dataset"] = df.to_dict()

	# Add the list of all tokens to the dictionary
	main_dict[lang]["token_overlap"] = {"overlap_percentage":overlap_per, "token_list": token_list, "overlap_token_list":overlap_token_list}

	# Add to the results
	token_overlap_results[lang] = {"percentage": overlap_per, "overlap_list_size": len(overlap_token_list), "overlap_set_size": len(set(overlap_token_list))}

tokenizer_config.json: 100%|██████████| 25.0/25.0 [00:00<00:00, 3.22kB/s]


Tokenizing text.


100%|██████████| 70/70 [00:00<00:00, 320.22it/s]


['▁Angel', 'o', '▁Che', 't', 'cuti', ',', '▁se', '▁j', 'kun', '▁qe']
All tokens:
33697
Calculating overlap.


100%|██████████| 33697/33697 [01:10<00:00, 477.93it/s]


...

Number of tokens that overlap: 21808
Number of different tokens that overlap: 4383
Percentage of overlap: 0.8215173660815189
Tokenizing text.


100%|██████████| 80/80 [00:00<00:00, 654.96it/s]


['▁Kita', 'jsko', '▁mesto', '▁duhov', '▁V', '▁Notranj', 'i', '▁Mongol', 'iji', '▁raste']
All tokens:
26292
Calculating overlap.


100%|██████████| 26292/26292 [00:54<00:00, 481.40it/s]

Number of tokens that overlap: 25616
Number of different tokens that overlap: 5281
Percentage of overlap: 0.9742887570363609


In [6]:
token_overlap_results = {}

for lang in list(main_dict.keys()):
	overlap_token_list = main_dict[lang]["token_overlap"]["overlap_token_list"]
	# Add to the results
	token_overlap_results[lang] = {"percentage": main_dict[lang]["token_overlap"]["overlap_percentage"], "overlap_list_size": len(overlap_token_list), "overlap_set_size": len(set(overlap_token_list))}

In [7]:
token_overlap_results

{'mt': {'percentage': 0.8022969403804493,
  'overlap_list_size': 27035,
  'overlap_set_size': 2771},
 'el': {'percentage': 0.1614276568501921,
  'overlap_list_size': 5043,
  'overlap_set_size': 822},
 'tr': {'percentage': 0.5208719382770122,
  'overlap_list_size': 15460,
  'overlap_set_size': 2412},
 'sq': {'percentage': 0.6038502030380508,
  'overlap_list_size': 16060,
  'overlap_set_size': 2732},
 'is': {'percentage': 0.5130030019900833,
  'overlap_list_size': 15209,
  'overlap_set_size': 2017},
 'uk': {'percentage': 0.15665821179454664,
  'overlap_list_size': 4941,
  'overlap_set_size': 408},
 'ca': {'percentage': 0.7448446122567529,
  'overlap_list_size': 20516,
  'overlap_set_size': 2897},
 'mk': {'percentage': 0.14598936285683273,
  'overlap_list_size': 4035,
  'overlap_set_size': 656},
 'hr': {'percentage': 0.8215173660815189,
  'overlap_list_size': 21808,
  'overlap_set_size': 4383},
 'sl': {'percentage': 0.9742887570363609,
  'overlap_list_size': 25616,
  'overlap_set_size': 5

In [8]:
overlap_df = pd.DataFrame(token_overlap_results).transpose().sort_values(by="percentage", ascending=False)
overlap_df

,percentage,overlap_list_size,overlap_set_size
sl,0.974289,25616.0,5281.0
hr,0.821517,21808.0,4383.0
mt,0.802297,27035.0,2771.0
ca,0.744845,20516.0,2897.0
sq,0.603850,16060.0,2732.0
tr,0.520872,15460.0,2412.0
is,0.513003,15209.0,2017.0
el,0.161428,5043.0,822.0
uk,0.156658,4941.0,408.0
mk,0.145989,4035.0,656.0


In [10]:
# Save the extended json dict
with open("manual-annotations/multilingual-genre-annotated-test-set.json", "w") as file:
	json.dump(main_dict, file)

Add token counts information

In [11]:
# Import the final dataset with test sets
with open("manual-annotations/multilingual-genre-annotated-test-set.json") as main_file:
	main_dict = json.load(main_file)

main_dict.keys()

dict_keys(['mt', 'el', 'tr', 'sq', 'is', 'uk', 'ca', 'mk', 'hr', 'sl'])

In [9]:
token_number = {}
type_number = {}

for lang in list(main_dict.keys()):
	#print(f"Creating token dict for {lang}")
	current_token_list = main_dict[lang]["token_overlap"]["token_list"]

	#print(f"No of tokens: {len(current_token_list)}")

	# Create a dictionary which counts the occurrences of the tokens

	#word_dict_test = Counter(current_token_list)

	# Sort the dictionary alphabetically based on keys
	#word_dict_test = dict(sorted(word_dict_test.items()))

	# Add information on no. of tokens and words to a dict
	token_number[lang] = len(current_token_list)
	#type_number[lang] = len(word_dict_test)
	type_number[lang] = len(set(current_token_list))

	#print(list(word_dict_test.items())[:100])
	#print(f"No of unique tokens: {len(word_dict_test)}")

	# Add the count of tokens to the main dictionary
	#main_dict[lang]["token_overlap"]["token_count"] = word_dict_test

# Create a dataframe for statistics
token_df = pd.DataFrame({"tokens": token_number, "types": type_number})
print(token_df.to_markdown())

|    |   tokens |   types |
|:---|---------:|--------:|
| mt |    33697 |    3874 |
| el |    31240 |    4751 |
| tr |    29681 |    6231 |
| sq |    26596 |    4871 |
| is |    29647 |    4522 |
| uk |    31540 |    6463 |
| ca |    27544 |    5314 |
| mk |    27639 |    5468 |
| hr |    26546 |    6222 |
| sl |    26292 |    5763 |


In [10]:
# Merge token_df and overlap_df
merged = pd.concat([overlap_df, token_df], axis=1)
merged

# Reorder columns

merged = merged[["percentage", "tokens", "overlap_list_size", "types","overlap_set_size"]]

# Rename columns

merged.columns = ["overlap_percentage", "all_tokens", "overlapping_tokens", "all_types", "overlapping_types"]

merged

,overlap_percentage,all_tokens,overlapping_tokens,all_types,overlapping_types
sl,0.974289,26292,25616.0,5763,5281.0
hr,0.821517,26546,21808.0,6222,4383.0
mt,0.802297,33697,27035.0,3874,2771.0
ca,0.744845,27544,20516.0,5314,2897.0
sq,0.603850,26596,16060.0,4871,2732.0
tr,0.520872,29681,15460.0,6231,2412.0
is,0.513003,29647,15209.0,4522,2017.0
el,0.161428,31240,5043.0,4751,822.0
uk,0.156658,31540,4941.0,6463,408.0
mk,0.145989,27639,4035.0,5468,656.0


In [13]:
print(merged.to_markdown())

|    |   overlap_percentage |   all_tokens |   overlapping_tokens |   all_types |   overlapping_types |
|:---|---------------------:|-------------:|---------------------:|------------:|--------------------:|
| sl |             0.974289 |        26292 |                25616 |        5763 |                5281 |
| hr |             0.821517 |        26546 |                21808 |        6222 |                4383 |
| mt |             0.802297 |        33697 |                27035 |        3874 |                2771 |
| ca |             0.744845 |        27544 |                20516 |        5314 |                2897 |
| sq |             0.60385  |        26596 |                16060 |        4871 |                2732 |
| tr |             0.520872 |        29681 |                15460 |        6231 |                2412 |
| is |             0.513003 |        29647 |                15209 |        4522 |                2017 |
| el |             0.161428 |        31240 |                 504

In [21]:
# Let's see the most frequent tokens
most_frequent = {}

# Sort the dictionary by values (word counts) in descending order
for lang in list(main_dict.keys()):
	token_list = main_dict[lang]["token_overlap"]["overlap_token_list"]

	word_dict = Counter(token_list)

	most_frequent[lang] = word_dict.most_common(10)

print(pd.DataFrame({"most_frequent_type": most_frequent}).to_markdown())

|    | most_frequent_type                                                                                                                     |
|:---|:---------------------------------------------------------------------------------------------------------------------------------------|
| ca | [(',', 1079), ('▁de', 1013), ('.', 675), ('s', 651), ('▁i', 566), ('▁la', 560), ('▁a', 530), ('▁que', 439), ("'", 357), ('’', 334)]    |
| el | [('▁', 1017), ('.', 801), (',', 782), ('▁η', 159), ('▁"', 98), ('▁(', 83), (':', 62), (')', 60), ('"', 55), ('-', 45)]                 |
| hr | [(',', 878), ('.', 766), ('▁i', 546), ('▁u', 430), ('a', 413), ('▁je', 350), ('▁na', 282), ('▁za', 253), ('▁se', 239), ('e', 219)]     |
| is | [('.', 1021), ('▁og', 640), (',', 598), ('▁', 532), ('▁er', 357), ('s', 316), ('▁sem', 282), ('a', 272), ('i', 272), ('▁til', 261)]    |
| mk | [(',', 1021), ('.', 738), ('▁', 341), ('o', 126), ('e', 96), ('▁"', 85), (':', 64), ('"', 53), ('-', 47), ('▁-', 42)]            

In [11]:
# Let's see the most frequent tokens
most_frequent = {}
# Sort the dictionary by values (word counts) in descending order
for lang in list(main_dict.keys()):
	most_frequent[lang] = (sorted(main_dict[lang]['token_overlap']['token_count'].items(), key=lambda x: x[1], reverse=True)[:10])

print(pd.DataFrame({"most_frequent_type": most_frequent}).to_markdown())

KeyError: 'token_count'

In [36]:
# Save the main dict
with open("manual-annotations/multilingual-genre-annotated-test-set.json", "w") as file:
	json.dump(main_dict, file)

# Compare token overlap on label level 

### Create label-level token counts for train dataset

In [12]:
# Open the tokenized train df
train_df = pd.read_json("datasets/tokenized_datasets/X-GENRE-train-tokenized.json")
train_df.head(2)

,text,labels,dataset,language,tokens_train,token_ids,text_norm,tokens_train_norm
0,"Seeking All Things Brilliant ""I want people to...",Other,CORE,English,"[▁See, king, ▁All, ▁Things, ▁Br, illian, t, ▁""...","[6872, 6048, 3164, 119175, 13008, 162076, 18, ...","seeking all things brilliant ""i want people to...","[▁seeking, ▁all, ▁things, ▁brilliant, ▁"", i, ▁..."
1,Meet Orchid du Bois I first met Hayley Mowday ...,Other,CORE,English,"[▁Meet, ▁Or, ch, id, ▁du, ▁Bo, is, ▁I, ▁first,...","[72626, 3347, 206, 532, 115, 2460, 164, 87, 51...",meet orchid du bois i first met hayley mowday ...,"[▁meet, ▁or, ch, id, ▁du, ▁bois, ▁i, ▁first, ▁..."


In [13]:
train_df.labels.unique()

array(['Other', 'Information/Explanation', 'News', 'Instruction',
       'Opinion/Argumentation', 'Forum', 'Prose/Lyrical', 'Legal',
       'Promotion'], dtype=object)

In [14]:
# Separate the train df into label-based dfs

# Create lists of tokens that are label based
label_tokens = {}
token_count = {}
type_count = {}

for label in ['Information/Explanation', 'News', 'Instruction','Opinion/Argumentation', 'Forum', 'Prose/Lyrical', 'Legal','Promotion']:
	print(f"Processing {label}")

	label_df = train_df[train_df["labels"] == label]
	display(label_df.head(3))

	# Create a list of tokens, where we take only the first 512 tokens
	train_tokens_shortened = []

	for i in label_df["tokens_train"].to_list():
		train_tokens_shortened.extend(i[:512])

	print(f"Number of all tokens: {len(train_tokens_shortened)}")

	# Add to dictionaries
	label_tokens[label] = train_tokens_shortened
	token_count[label] = len(train_tokens_shortened)
	type_count[label] = len(set(train_tokens_shortened))

Processing Information/Explanation


,text,labels,dataset,language,tokens_train,token_ids,text_norm,tokens_train_norm
2,Abstract Objective: Reporting bias due to soci...,Information/Explanation,CORE,English,"[▁Abstract, ▁Object, ive, :, ▁Report, ing, ▁bi...","[233973, 134549, 5844, 12, 34798, 214, 333, 16...",abstract objective: reporting bias due to soci...,"[▁abstract, ▁objective, :, ▁report, ing, ▁bi, ..."
3,In 2009 the song was the focus of a successful...,Information/Explanation,CORE,English,"[▁In, ▁2009, ▁the, ▁song, ▁was, ▁the, ▁focus, ...","[360, 1877, 70, 11531, 509, 70, 32153, 111, 10...",in 2009 the song was the focus of a successful...,"[▁in, ▁2009, ▁the, ▁song, ▁was, ▁the, ▁focus, ..."
39,Story: Whaling Page 4 -- M?ori and whaling Wha...,Information/Explanation,CORE,English,"[▁Story, :, ▁W, hal, ing, ▁Page, ▁4, ▁--, ▁M, ...","[30575, 12, 601, 4200, 214, 14231, 201, 4210, ...",story: whaling page 4 -- m?ori and whaling wha...,"[▁story, :, ▁w, hal, ing, ▁page, ▁4, ▁--, ▁m, ..."


Number of all tokens: 124130
Processing News


,text,labels,dataset,language,tokens_train,token_ids,text_norm,tokens_train_norm
4,QuotW This was the week when neither rumours o...,News,CORE,English,"[▁Quo, t, W, ▁This, ▁was, ▁the, ▁week, ▁when, ...","[43851, 18, 1456, 3293, 509, 70, 5895, 3229, 2...",quotw this was the week when neither rumours o...,"[▁quo, tw, ▁this, ▁was, ▁the, ▁week, ▁when, ▁n..."
5,KaZaA claims it can't stop users sharing music...,News,CORE,English,"[▁Ka, Za, A, ▁claims, ▁it, ▁can, ', t, ▁stop, ...","[1136, 16737, 284, 140526, 442, 831, 25, 18, 7...",kazaa claims it can't stop users sharing music...,"[▁kaza, a, ▁claims, ▁it, ▁can, ', t, ▁stop, ▁u..."
9,Nebraska fans checking out airfare for a trip ...,News,CORE,English,"[▁Ne, bra, ska, ▁fans, ▁checking, ▁out, ▁air, ...","[799, 2844, 937, 35992, 175199, 1810, 1831, 44...",nebraska fans checking out airfare for a trip ...,"[▁ne, bra, ska, ▁fans, ▁checking, ▁out, ▁air, ..."


Number of all tokens: 136557
Processing Instruction


...

Number of all tokens: 46860
Processing Legal


,text,labels,dataset,language,tokens_train,token_ids,text_norm,tokens_train_norm
35,Full Terms and Conditions Eligibility to enter...,Legal,CORE,English,"[▁Full, ▁Terms, ▁and, ▁Condi, tions, ▁E, ligi,...","[9312, 165504, 136, 46347, 5256, 241, 7883, 83...",full terms and conditions eligibility to enter...,"[▁full, ▁terms, ▁and, ▁conditions, ▁e, ligi, b..."
110,Commonwealth Consolidated Acts INCOME TAX ASSE...,Legal,CORE,English,"[▁Common, we, al, th, ▁Con, solid, ated, ▁Act,...","[151301, 1177, 289, 927, 1657, 97281, 27686, 2...",commonwealth consolidated acts income tax asse...,"[▁common, we, al, th, ▁consolida, ted, ▁act, s..."
122,"This Terms of Use Agreement (""Agreement"") is b...",Legal,CORE,English,"[▁This, ▁Terms, ▁of, ▁Use, ▁Agreement, ▁("", A,...","[3293, 165504, 111, 36836, 186670, 24073, 284,...","this terms of use agreement (""agreement"") is b...","[▁this, ▁terms, ▁of, ▁use, ▁agreement, ▁("", a,..."


Number of all tokens: 28496
Processing Promotion


,text,labels,dataset,language,tokens_train,token_ids,text_norm,tokens_train_norm
79,Post navigation Citizen Fish are back on the r...,Promotion,CORE,English,"[▁Post, ▁navigation, ▁Citizen, ▁Fish, ▁are, ▁b...","[2795, 134470, 193223, 104796, 621, 4420, 98, ...",post navigation citizen fish are back on the r...,"[▁post, ▁navigation, ▁citizen, ▁fish, ▁are, ▁b..."
130,Win yourself a FREE copy of the BradyGames off...,Promotion,CORE,English,"[▁Win, ▁yourself, ▁a, ▁FREE, ▁copy, ▁of, ▁the,...","[17686, 31949, 10, 86697, 43658, 111, 70, 5859...",win yourself a free copy of the bradygames off...,"[▁win, ▁yourself, ▁a, ▁free, ▁copy, ▁of, ▁the,..."
179,Do You Want To Know The Quick Secret for On Pa...,Promotion,CORE,English,"[▁Do, ▁You, ▁Want, ▁To, ▁Know, ▁The, ▁Quick, ▁...","[984, 2583, 42335, 717, 70829, 581, 89038, 390...",do you want to know the quick secret for on pa...,"[▁do, ▁you, ▁want, ▁to, ▁know, ▁the, ▁quick, ▁..."


Number of all tokens: 88626


In [15]:
# Calculate numbers for each label
label_results_train = pd.DataFrame({"token_count": token_count, "type_count": type_count})

print(label_results_train.to_markdown())

|                         |   token_count |   type_count |
|:------------------------|--------------:|-------------:|
| Information/Explanation |        124130 |        14678 |
| News                    |        136557 |        15319 |
| Instruction             |         83750 |         8929 |
| Opinion/Argumentation   |        103141 |        13088 |
| Forum                   |         58900 |         8555 |
| Prose/Lyrical           |         46860 |         5990 |
| Legal                   |         28496 |         4425 |
| Promotion               |         88626 |        12548 |


In [16]:
# Save the label token count
with open("datasets/tokenized_datasets/X-GENRE-train-label-token-count.json", "w") as train_label_count_file:
	json.dump(label_tokens, train_label_count_file)

### Create label-level token counts for test sets

In [17]:
# Import the final dataset with test sets
with open("manual-annotations/multilingual-genre-annotated-test-set.json") as main_file:
	main_dict = json.load(main_file)

main_dict["uk"].keys()

dict_keys(['accuracy', 'micro_f1', 'macro_f1', 'label_scores', 'dataset', 'token_overlap'])

In [18]:
pd.DataFrame(main_dict["sl"]["dataset"]).head(2)

,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids
1,CLASSLA-web.sl.1087171,Opinion/Argumentation,Kitajsko mesto duhov\nV Notranji Mongoliji ras...,The Chinese ghost city in Inner Mongolia is gr...,"CLASSLA-web.sl.1087171', 'domain': 'mladina.si'}",News,"[▁Kita, jsko, ▁mesto, ▁duhov, ▁V, ▁Notranj, i,...","[12992, 30878, 22041, 87909, 310, 240213, 14, ..."
2,CLASSLA-web.sl.1215246,Instruction,"Krompir skuhamo, olupimo in narežemo na tanke ...","Cook the potatoes, peel and cut into thin ring...","CLASSLA-web.sl.1215246', 'domain': 'emorje.com'}",Instruction,"[▁Krom, pir, ▁s, kuha, mo, ,, ▁olup, imo, ▁in,...","[128458, 21738, 91, 60863, 432, 4, 37663, 2414..."


In [19]:
# Do the same as with the dataset, but on every language
lang_results = {}

for lang in list(main_dict.keys()):
	print(f"Processing {lang}")

	label_token_dict = {}

	# Current df
	df = pd.DataFrame(main_dict[lang]["dataset"])
	display(df.head(2))

	for label in ['Information/Explanation', 'News', 'Instruction','Opinion/Argumentation', 'Forum', 'Prose/Lyrical', 'Legal','Promotion']:
		print(f"Processing {label}")

		label_df = df[df["y_true"] == label]

		# Create a list of tokens
		token_list = []

		for i in label_df["tokens"].to_list():
			token_list.extend(i)

		# Add to dictionary
		label_token_dict[label] = token_list

	# Add to main dict
	main_dict[lang]["token_overlap"]["label_level_token_lists"] = label_token_dict


Processing mt


,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids
0,macocu.mt.402244,News,"Angelo Chetcuti, se jkun qed jieħu post Bjorn ...","Angelo Chetcuti, will be replacing Bjorn Vassa...",{'text_id': 'macocu.mt.402244'},News,"[▁Angel, o, ▁Che, t, cuti, ,, ▁se, ▁j, kun, ▁q...","[26902, 31, 5024, 18, 64969, 4, 40, 1647, 6262..."
1,macocu.mt.377203,Prose/Lyrical,Poltergeist jirreferi għal fenomeni oħra tal-m...,"Poltergeist refers to other woman's phenomena,...",{'text_id': 'macocu.mt.377203'},Opinion/Argumentation,"[▁Pol, ter, geist, ▁jir, re, feri, ▁g, ħ, al, ...","[9017, 720, 178490, 52826, 107, 26926, 706, 24..."


Processing Information/Explanation
Processing News
Processing Instruction
Processing Opinion/Argumentation
Processing Forum
Processing Prose/Lyrical
Processing Legal
Processing Promotion
Processing el


,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids
0,macocu.el.1525713,Instruction,Ενημέρωση του Pegasus Estiasi με τις εισερχόμε...,Update Pegasus Estiasi with Incoming Calls\n\n...,{'text_id': 'macocu.el.1525713'},Instruction,"[▁Ενημέρωση, ▁του, ▁Pegasus, ▁Esti, asi, ▁με, ...","[236422, 385, 241060, 60271, 1544, 558, 1713, ..."
1,macocu.el.3525724,Forum,Η τιμή της έκδοσης 8GB/ 128GB είναι 1.299 ευρώ...,"The price of 8GB/ 128GB is € 1,299, of the 12G...",{'text_id': 'macocu.el.3525724'},Forum,"[▁Η, ▁τιμή, ▁της, ▁έκδοση, ς, ▁8, GB, /, ▁128,...","[1700, 77118, 463, 110873, 235, 382, 8359, 64,..."


Processing Information/Explanation
Processing News
Processing Instruction
Processing Opinion/Argumentation
Processing Forum
Processing Prose/Lyrical
Processing Legal
Processing Promotion
Processing tr


...

Processing Information/Explanation
Processing News
Processing Instruction
Processing Opinion/Argumentation
Processing Forum
Processing Prose/Lyrical
Processing Legal
Processing Promotion
Processing hr


,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids
0,CLASSLA-web.hr.1033815,Promotion,"O proizvodu\nColor Transformer, za pametno i j...","About the Color Transformer product, for smart...","CLASSLA-web.hr.1033815', 'domain': 'hairshop.hr'}",Promotion,"[▁O, ▁proizvod, u, ▁Color, ▁Trans, former, ,, ...","[180, 43170, 34, 51193, 11062, 82772, 4, 80, 6..."
2,CLASSLA-web.hr.1119579,Promotion,Sunčano selo / Sunny village\nNa obroncima Bil...,Sunshine / Sunny Village on the slopes of Bilo...,"CLASSLA-web.hr.1119579', 'domain': 'vikendi.com'}",Promotion,"[▁Sun, čan, o, ▁se, lo, ▁/, ▁Sunny, ▁village, ...","[7550, 17129, 31, 40, 365, 248, 151197, 54427,..."


Processing Information/Explanation
Processing News
Processing Instruction
Processing Opinion/Argumentation
Processing Forum
Processing Prose/Lyrical
Processing Legal
Processing Promotion
Processing sl


,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids
1,CLASSLA-web.sl.1087171,Opinion/Argumentation,Kitajsko mesto duhov\nV Notranji Mongoliji ras...,The Chinese ghost city in Inner Mongolia is gr...,"CLASSLA-web.sl.1087171', 'domain': 'mladina.si'}",News,"[▁Kita, jsko, ▁mesto, ▁duhov, ▁V, ▁Notranj, i,...","[12992, 30878, 22041, 87909, 310, 240213, 14, ..."
2,CLASSLA-web.sl.1215246,Instruction,"Krompir skuhamo, olupimo in narežemo na tanke ...","Cook the potatoes, peel and cut into thin ring...","CLASSLA-web.sl.1215246', 'domain': 'emorje.com'}",Instruction,"[▁Krom, pir, ▁s, kuha, mo, ,, ▁olup, imo, ▁in,...","[128458, 21738, 91, 60863, 432, 4, 37663, 2414..."


Processing Information/Explanation
Processing News
Processing Instruction
Processing Opinion/Argumentation
Processing Forum
Processing Prose/Lyrical
Processing Legal
Processing Promotion


In [25]:
# Save the extended json dict
with open("manual-annotations/multilingual-genre-annotated-test-set.json", "w") as file:
	json.dump(main_dict, file)

In [27]:
main_dict["sl"]['token_overlap'].keys()

dict_keys(['overlap_percentage', 'token_list', 'overlap_token_list', 'label_level_token_lists'])

In [24]:
main_dict["sl"]["token_overlap"]["label_level_token_lists"]

{'Information/Explanation': ['▁D',
  'vor',
  'ec',
  '▁je',
  '▁morda',
  '▁na',
  'sled',
  'nik',
  '▁enega',
  '▁od',
  '▁številnih',
  '▁v',
  '▁vir',
  'ih',
  '▁iz',
  'pri',
  'ča',
  'nih',
  '▁srednje',
  've',
  'ških',
  '▁dvor',
  'ov',
  '▁na',
  '▁šir',
  'š',
  'em',
  '▁območju',
  '▁Vip',
  'ave',
  '.',
  '▁Prvi',
  'č',
  '▁je',
  '▁bil',
  '▁omen',
  'jen',
  '▁leta',
  '▁16',
  '30',
  '▁v',
  '▁popis',
  'u',
  '▁za',
  'pušč',
  'ine',
  '▁Balta',
  'zar',
  'ja',
  '▁pl',
  '.',
  '▁Raum',
  'sch',
  'üs',
  's',
  'la',
  ',',
  '▁gospod',
  'a',
  '▁na',
  '▁Bel',
  'ne',
  'ku',
  '▁pri',
  '▁Morav',
  'ča',
  'h',
  '▁na',
  '▁Gor',
  'en',
  'jskem',
  '▁in',
  '▁v',
  '▁Vrh',
  'pol',
  'ju',
  '▁pri',
  '▁Vip',
  'avi',
  '.',
  '▁Zaradi',
  '▁lege',
  '▁ob',
  '▁vodi',
  '▁je',
  '▁do',
  'bil',
  '▁nem',
  'ško',
  '▁ime',
  '▁Schön',
  'au',
  ',',
  '▁ki',
  '▁pomeni',
  '▁Le',
  'pi',
  '▁log',
  '.',
  '▁Popis',
  '▁njegove',
  '▁za',
  'pušč',
  '

## Calculate overlap on label level

In [28]:
# Open label-based train token count
with open("datasets/tokenized_datasets/X-GENRE-train-label-token-count.json", "r") as train_label_count_file:
	label_token_count_train = json.load(train_label_count_file)

label_token_count_train.keys()

dict_keys(['Information/Explanation', 'News', 'Instruction', 'Opinion/Argumentation', 'Forum', 'Prose/Lyrical', 'Legal', 'Promotion'])

In [29]:
# Import the main dict for test sets
with open("manual-annotations/multilingual-genre-annotated-test-set.json", "r") as file:
	main_dict = json.load(file)

main_dict["sl"]["token_overlap"]["label_level_token_lists"]

{'Information/Explanation': ['▁D',
  'vor',
  'ec',
  '▁je',
  '▁morda',
  '▁na',
  'sled',
  'nik',
  '▁enega',
  '▁od',
  '▁številnih',
  '▁v',
  '▁vir',
  'ih',
  '▁iz',
  'pri',
  'ča',
  'nih',
  '▁srednje',
  've',
  'ških',
  '▁dvor',
  'ov',
  '▁na',
  '▁šir',
  'š',
  'em',
  '▁območju',
  '▁Vip',
  'ave',
  '.',
  '▁Prvi',
  'č',
  '▁je',
  '▁bil',
  '▁omen',
  'jen',
  '▁leta',
  '▁16',
  '30',
  '▁v',
  '▁popis',
  'u',
  '▁za',
  'pušč',
  'ine',
  '▁Balta',
  'zar',
  'ja',
  '▁pl',
  '.',
  '▁Raum',
  'sch',
  'üs',
  's',
  'la',
  ',',
  '▁gospod',
  'a',
  '▁na',
  '▁Bel',
  'ne',
  'ku',
  '▁pri',
  '▁Morav',
  'ča',
  'h',
  '▁na',
  '▁Gor',
  'en',
  'jskem',
  '▁in',
  '▁v',
  '▁Vrh',
  'pol',
  'ju',
  '▁pri',
  '▁Vip',
  'avi',
  '.',
  '▁Zaradi',
  '▁lege',
  '▁ob',
  '▁vodi',
  '▁je',
  '▁do',
  'bil',
  '▁nem',
  'ško',
  '▁ime',
  '▁Schön',
  'au',
  ',',
  '▁ki',
  '▁pomeni',
  '▁Le',
  'pi',
  '▁log',
  '.',
  '▁Popis',
  '▁njegove',
  '▁za',
  'pušč',
  '

In [31]:
# Improved code to calculate token overlap - we can simply calculate which tokens do not overlap - meaning that they occur only in one of the dataset, and calculate the percentage overlap based on that
token_overlap_label_results = {}
results = {}

# Loop through the datasets and labels and calculate token overlap
for lang in list(main_dict.keys()):
	print(lang)
	# Create a dict for labels
	label_overlap = {}
	label_overlap_tokens = {}

	# loop through labels
	if lang != "mt":
		for label in ['Information/Explanation', 'News', 'Instruction','Opinion/Argumentation', 'Forum', 'Prose/Lyrical', 'Legal','Promotion']:
			print(label)
			token_list_test = main_dict[lang]["token_overlap"]["label_level_token_lists"][label]

			token_list_train = label_token_count_train[label]

			# See how many tokens do not overlap
			no_overlap_counter = 0

			# Save tokens that overlap for further inspection
			overlap_token_list = []

			# We calculate overlap by counting how many tokens do not appear in both sets
			for token in tqdm(token_list_test):
				if token not in token_list_train:
					no_overlap_counter += 1
				else:
					overlap_token_list.append(token)

			# Out of all tokens in test set, how many do not overlap with train set?
			no_overlap_per = no_overlap_counter/len(token_list_test)

			# Calculate percentage of overlap based on that
			overlap_per = 1-no_overlap_per

			print(f"Percentage of overlap: {overlap_per}")

			label_overlap[label] = overlap_per
			label_overlap_tokens[label] = overlap_token_list

			# Add to the results
			results[f"{lang}-{label}"] = overlap_per

		# Add the list of all tokens to the dictionary
		main_dict[lang]["token_overlap"]["label_overlap_percentage"] = label_overlap
		main_dict[lang]["token_overlap"]["label_overlap_token_list"] = label_overlap_tokens
	else:
		# Skip "Legal" for Maltese, because we do not have this label there
		for label in ['Information/Explanation', 'News', 'Instruction','Opinion/Argumentation', 'Forum', 'Prose/Lyrical', 'Promotion']:
			print(label)
			token_list_test = main_dict[lang]["token_overlap"]["label_level_token_lists"][label]

			token_list_train = label_token_count_train[label]

			# See how many tokens do not overlap
			no_overlap_counter = 0

			# Save tokens that overlap for further inspection
			overlap_token_list = []

			# We calculate overlap by counting how many tokens do not appear in both sets
			for token in tqdm(token_list_test):
				if token not in token_list_train:
					no_overlap_counter += 1
				else:
					overlap_token_list.append(token)

			# Out of all tokens in test set, how many do not overlap with train set?
			no_overlap_per = no_overlap_counter/len(token_list_test)

			# Calculate percentage of overlap based on that
			overlap_per = 1-no_overlap_per

			print(f"Percentage of overlap: {overlap_per}")

			label_overlap[label] = overlap_per
			label_overlap_tokens[label] = overlap_token_list

			# Add to the results
			results[f"{lang}-{label}"] = overlap_per

		# Add the list of all tokens to the dictionary
		main_dict[lang]["token_overlap"]["label_overlap_percentage"] = label_overlap
		main_dict[lang]["token_overlap"]["label_overlap_token_list"] = label_overlap_tokens		

mt
Information/Explanation


100%|██████████| 6656/6656 [00:03<00:00, 2089.15it/s]


Percentage of overlap: 0.7145432692307692
News


100%|██████████| 6755/6755 [00:03<00:00, 1823.71it/s]


Percentage of overlap: 0.7083641746854182
Instruction


...

Percentage of overlap: 0.6995531110347198
Legal


100%|██████████| 3446/3446 [00:00<00:00, 11372.51it/s]


Percentage of overlap: 0.7556587347649448
Promotion


100%|██████████| 3203/3203 [00:00<00:00, 6392.10it/s]

Percentage of overlap: 0.8741804558226662


In [32]:
results

{'mt-Information/Explanation': 0.7145432692307692,
 'mt-News': 0.7083641746854182,
 'mt-Instruction': 0.6194013781223083,
 'mt-Opinion/Argumentation': 0.68212890625,
 'mt-Forum': 0.583984375,
 'mt-Prose/Lyrical': 0.591796875,
 'mt-Promotion': 0.6578768288533514,
 'el-Information/Explanation': 0.13908701854493577,
 'el-News': 0.1162524992859183,
 'el-Instruction': 0.16122517670817904,
 'el-Opinion/Argumentation': 0.11699661137863382,
 'el-Forum': 0.17033287911232242,
 'el-Prose/Lyrical': 0.14589136490250698,
 'el-Legal': 0.1395486935866983,
 'el-Promotion': 0.15900735294117652,
 'tr-Information/Explanation': 0.47924858016601135,
 'tr-News': 0.44105326152004787,
 'tr-Instruction': 0.3827071290944123,
 'tr-Opinion/Argumentation': 0.4332798859789774,
 'tr-Forum': 0.44330537433985706,
 'tr-Prose/Lyrical': 0.41339001062699254,
 'tr-Legal': 0.3146586891487626,
 'tr-Promotion': 0.39420689655172414,
 'sq-Information/Explanation': 0.4844110854503464,
 'sq-News': 0.4650103519668737,
 'sq-Instruct

In [33]:
# Inspect the results
overlap_df = pd.DataFrame({"label": list(results.keys()), "overlap": list(results.values())})
overlap_df

,label,overlap
0,mt-Information/Explanation,0.714543
1,mt-News,0.708364
2,mt-Instruction,0.619401
3,mt-Opinion/Argumentation,0.682129
4,mt-Forum,0.583984
...,...,...
74,sl-Opinion/Argumentation,0.888709
75,sl-Forum,0.801008
76,sl-Prose/Lyrical,0.699553
77,sl-Legal,0.755659


In [34]:
# Save the label-level-overlap
with open("datasets/label-level-token-overlap.csv", "w") as file:
	overlap_df.to_csv(file)

In [35]:
# Save the extended json dict
with open("manual-annotations/multilingual-genre-annotated-test-set.json", "w") as file:
	json.dump(main_dict, file)

# English word overlap

## Prepare the train dataset

In [114]:
# Open the tokenized df
train_df = pd.read_json("datasets/tokenized_datasets/X-GENRE-train-tokenized-old.json")
train_df.head(2)

,text,labels,dataset,language,tokens_train,token_ids,text_norm,tokens_train_norm
0,"Seeking All Things Brilliant ""I want people to...",Other,CORE,English,"[▁See, king, ▁All, ▁Things, ▁Br, illian, t, ▁""...","[6872, 6048, 3164, 119175, 13008, 162076, 18, ...","seeking all things brilliant ""i want people to...","[▁seeking, ▁all, ▁things, ▁brilliant, ▁"", i, ▁..."
1,Meet Orchid du Bois I first met Hayley Mowday ...,Other,CORE,English,"[▁Meet, ▁Or, ch, id, ▁du, ▁Bo, is, ▁I, ▁first,...","[72626, 3347, 206, 532, 115, 2460, 164, 87, 51...",meet orchid du bois i first met hayley mowday ...,"[▁meet, ▁or, ch, id, ▁du, ▁bois, ▁i, ▁first, ▁..."


In [115]:
# Add lists of English words
token_list = train_df["tokens_train"].to_list()

all_text_word_list = []

for text in token_list:

	word_list = []

	# Take only the first 512 tokens
	for word in text[:512]:
		for punct in [".", '"', "!", "?", ":", ";", ",", "/", ")", "(", "[", "]"]:
			word = word.replace(punct, "").lower()

		if len(word) > 0:
			if word[0] == "▁":
				word_list.append(word.replace("▁", ""))
			else:
				previous_word = word_list[-1]
				word_list[-1] = previous_word + word

	all_text_word_list.append(word_list)

train_df["word_list"] = all_text_word_list
train_df.head()


,text,labels,dataset,language,tokens_train,token_ids,text_norm,tokens_train_norm,word_list
0,"Seeking All Things Brilliant ""I want people to...",Other,CORE,English,"[▁See, king, ▁All, ▁Things, ▁Br, illian, t, ▁""...","[6872, 6048, 3164, 119175, 13008, 162076, 18, ...","seeking all things brilliant ""i want people to...","[▁seeking, ▁all, ▁things, ▁brilliant, ▁"", i, ▁...","[seeking, all, things, brilliant, i, want, peo..."
1,Meet Orchid du Bois I first met Hayley Mowday ...,Other,CORE,English,"[▁Meet, ▁Or, ch, id, ▁du, ▁Bo, is, ▁I, ▁first,...","[72626, 3347, 206, 532, 115, 2460, 164, 87, 51...",meet orchid du bois i first met hayley mowday ...,"[▁meet, ▁or, ch, id, ▁du, ▁bois, ▁i, ▁first, ▁...","[meet, orchid, du, bois, i, first, met, hayley..."
2,Abstract Objective: Reporting bias due to soci...,Information/Explanation,CORE,English,"[▁Abstract, ▁Object, ive, :, ▁Report, ing, ▁bi...","[233973, 134549, 5844, 12, 34798, 214, 333, 16...",abstract objective: reporting bias due to soci...,"[▁abstract, ▁objective, :, ▁report, ing, ▁bi, ...","[abstract, objective, reporting, bias, due, to..."
3,In 2009 the song was the focus of a successful...,Information/Explanation,CORE,English,"[▁In, ▁2009, ▁the, ▁song, ▁was, ▁the, ▁focus, ...","[360, 1877, 70, 11531, 509, 70, 32153, 111, 10...",in 2009 the song was the focus of a successful...,"[▁in, ▁2009, ▁the, ▁song, ▁was, ▁the, ▁focus, ...","[in, 2009, the, song, was, the, focus, of, a, ..."
4,QuotW This was the week when neither rumours o...,News,CORE,English,"[▁Quo, t, W, ▁This, ▁was, ▁the, ▁week, ▁when, ...","[43851, 18, 1456, 3293, 509, 70, 5895, 3229, 2...",quotw this was the week when neither rumours o...,"[▁quo, tw, ▁this, ▁was, ▁the, ▁week, ▁when, ▁n...","[quotw, this, was, the, week, when, neither, r..."


Add the MT-ed translation of GINCO

In [116]:
# Add the MT English translation of GINCO
mt_ginco = pd.read_csv("datasets/GINCO-MT-GINCO-keeptext-with-all-information.csv", sep="\t", index_col=0)
mt_ginco = mt_ginco[['Slovene_text', 'MT_text',]]
mt_ginco.rename(columns={"Slovene_text":"text"}, inplace=True)
mt_ginco

,text,MT_text
0,"Šport <p/> Zimska liga malega nogometa sobota,...",Sport <p/> Winter Little League Football Satur...
1,JEDILNIK <p/> Iskalnik <p/> Poglavitni cilj pr...,JEDILNIK <p/> Search <p/> The main objective o...
2,Projekt INNOVAge in zavod Oreli <p/> Zavod Ore...,Project INNOVAge and the Oreli Institute <p/> ...
3,"V novembru, mesecu preprečevanja odvisnosti, b...","In November, the month of addiction prevention..."
4,Selena Gomez ponudila v poslušanje novi album ...,Selena Gomez launches new album <p/> 16.07.201...
...,...,...
997,Projektne novine <p/> Promocijski projektni ča...,Project News <p/> Promotional project newspape...
998,V raznoliki ponudbi tušev izberite popolno raz...,Choose the perfect shower to match your taste ...
999,"O izdelku Za znamko Dame stojita dve ženski, z...",About the product There are two women behind t...
1000,Razprava pogosto potegne na plano najprej tist...,The debate often brings to the surface first t...


In [117]:
# Create a list of English words
# Add lists of English words - to do this properly, we would need to tokenize the English translation
token_list = mt_ginco["MT_text"].to_list()

all_text_word_list = []

for text in token_list:

	text = text.replace("<p/>", "")

	text_list = text.split(" ")

	word_list = []

	# Take only the first 512 words
	for word in text_list[:512]:
		for punct in [".", '"', "!", "?", ":", ";", ",", "/", ")", "(", "[", "]"]:
			word = word.replace(punct, "").lower()

		if len(word) > 0:
			word_list.append(word)

	all_text_word_list.append(word_list)

mt_ginco["word_list"] = all_text_word_list
mt_ginco.head()

,text,MT_text,word_list
0,"Šport <p/> Zimska liga malega nogometa sobota,...",Sport <p/> Winter Little League Football Satur...,"[sport, winter, little, league, football, satu..."
1,JEDILNIK <p/> Iskalnik <p/> Poglavitni cilj pr...,JEDILNIK <p/> Search <p/> The main objective o...,"[jedilnik, search, the, main, objective, of, t..."
2,Projekt INNOVAge in zavod Oreli <p/> Zavod Ore...,Project INNOVAge and the Oreli Institute <p/> ...,"[project, innovage, and, the, oreli, institute..."
3,"V novembru, mesecu preprečevanja odvisnosti, b...","In November, the month of addiction prevention...","[in, november, the, month, of, addiction, prev..."
4,Selena Gomez ponudila v poslušanje novi album ...,Selena Gomez launches new album <p/> 16.07.201...,"[selena, gomez, launches, new, album, 16072013..."


In [118]:
train_df_merged = pd.merge(left=train_df, right= mt_ginco, on="text", how="left")
train_df_merged[train_df_merged["dataset"] == "GINCO"].head(3)

,text,labels,dataset,language,tokens_train,token_ids,text_norm,tokens_train_norm,word_list_x,MT_text,word_list_y
607,Žakardska tkanina je splošna oznaka za vzorčas...,Information/Explanation,GINCO,Slovenian,"[▁Ža, kar, d, ska, ▁tkanin, a, ▁je, ▁spl, oš, ...","[78724, 1153, 71, 937, 124424, 11, 55, 27095, ...",zakardska tkanina je splosna oznaka za vzorcas...,"[▁zak, ard, ska, ▁tkanin, a, ▁je, ▁spl, os, na...","[žakardska, tkanina, je, splošna, oznaka, za, ...",Jacquard fabric is a general term for patterne...,"[jacquard, fabric, is, a, general, term, for, ..."
608,Kako nastane očka? <p/> Očetovstvo in materins...,Opinion/Argumentation,GINCO,Slovenian,"[▁Kako, ▁nastane, ▁o, čka, ?, ▁<, p, /, >, ▁O,...","[8438, 165103, 36, 5536, 32, 4426, 254, 64, 27...",kako nastane ocka? <p/> ocetovstvo in materins...,"[▁kako, ▁nastane, ▁o, cka, ?, ▁<, p, /, >, ▁o,...","[kako, nastane, očka, <p>, očetovstvo, in, mat...",How does Daddy come into being? <p/>Fatherhood...,"[how, does, daddy, come, into, being, fatherho..."
609,NOGOMETNO IGRIŠČE <p/> Goooolll ... Naši vodij...,Promotion,GINCO,Slovenian,"[▁NO, GO, MET, NO, ▁I, GRI, Š, ČE, ▁<, p, /, >...","[9520, 19930, 33677, 8575, 87, 191357, 6200, 7...",nogometno igrisce <p/> goooolll ... nasi vodij...,"[▁nogomet, no, ▁igri, s, ce, ▁<, p, /, >, ▁go,...","[nogometno, igrišče, <p>, goooolll, , naši, vo...",FOOTBALL PLAYING FIELD <p/> Goooolll ... Our t...,"[football, playing, field, goooolll, our, team..."


In [123]:
# Merge wordlists
train_df_merged.fillna(value=0, inplace=True)

train_df_merged["words"] = np.where(train_df_merged["word_list_y"] == 0, train_df_merged["word_list_x"], train_df_merged["word_list_y"])

display(train_df_merged.head(3))

display(train_df_merged[train_df_merged["dataset"] == "GINCO"].head(3))

,text,labels,dataset,language,tokens_train,token_ids,text_norm,tokens_train_norm,word_list_x,MT_text,word_list_y,words
0,"Seeking All Things Brilliant ""I want people to...",Other,CORE,English,"[▁See, king, ▁All, ▁Things, ▁Br, illian, t, ▁""...","[6872, 6048, 3164, 119175, 13008, 162076, 18, ...","seeking all things brilliant ""i want people to...","[▁seeking, ▁all, ▁things, ▁brilliant, ▁"", i, ▁...","[seeking, all, things, brilliant, i, want, peo...",0,0,"[seeking, all, things, brilliant, i, want, peo..."
1,Meet Orchid du Bois I first met Hayley Mowday ...,Other,CORE,English,"[▁Meet, ▁Or, ch, id, ▁du, ▁Bo, is, ▁I, ▁first,...","[72626, 3347, 206, 532, 115, 2460, 164, 87, 51...",meet orchid du bois i first met hayley mowday ...,"[▁meet, ▁or, ch, id, ▁du, ▁bois, ▁i, ▁first, ▁...","[meet, orchid, du, bois, i, first, met, hayley...",0,0,"[meet, orchid, du, bois, i, first, met, hayley..."
2,Abstract Objective: Reporting bias due to soci...,Information/Explanation,CORE,English,"[▁Abstract, ▁Object, ive, :, ▁Report, ing, ▁bi...","[233973, 134549, 5844, 12, 34798, 214, 333, 16...",abstract objective: reporting bias due to soci...,"[▁abstract, ▁objective, :, ▁report, ing, ▁bi, ...","[abstract, objective, reporting, bias, due, to...",0,0,"[abstract, objective, reporting, bias, due, to..."


,text,labels,dataset,language,tokens_train,token_ids,text_norm,tokens_train_norm,word_list_x,MT_text,word_list_y,words
607,Žakardska tkanina je splošna oznaka za vzorčas...,Information/Explanation,GINCO,Slovenian,"[▁Ža, kar, d, ska, ▁tkanin, a, ▁je, ▁spl, oš, ...","[78724, 1153, 71, 937, 124424, 11, 55, 27095, ...",zakardska tkanina je splosna oznaka za vzorcas...,"[▁zak, ard, ska, ▁tkanin, a, ▁je, ▁spl, os, na...","[žakardska, tkanina, je, splošna, oznaka, za, ...",Jacquard fabric is a general term for patterne...,"[jacquard, fabric, is, a, general, term, for, ...","[jacquard, fabric, is, a, general, term, for, ..."
608,Kako nastane očka? <p/> Očetovstvo in materins...,Opinion/Argumentation,GINCO,Slovenian,"[▁Kako, ▁nastane, ▁o, čka, ?, ▁<, p, /, >, ▁O,...","[8438, 165103, 36, 5536, 32, 4426, 254, 64, 27...",kako nastane ocka? <p/> ocetovstvo in materins...,"[▁kako, ▁nastane, ▁o, cka, ?, ▁<, p, /, >, ▁o,...","[kako, nastane, očka, <p>, očetovstvo, in, mat...",How does Daddy come into being? <p/>Fatherhood...,"[how, does, daddy, come, into, being, fatherho...","[how, does, daddy, come, into, being, fatherho..."
609,NOGOMETNO IGRIŠČE <p/> Goooolll ... Naši vodij...,Promotion,GINCO,Slovenian,"[▁NO, GO, MET, NO, ▁I, GRI, Š, ČE, ▁<, p, /, >...","[9520, 19930, 33677, 8575, 87, 191357, 6200, 7...",nogometno igrisce <p/> goooolll ... nasi vodij...,"[▁nogomet, no, ▁igri, s, ce, ▁<, p, /, >, ▁go,...","[nogometno, igrišče, <p>, goooolll, , naši, vo...",FOOTBALL PLAYING FIELD <p/> Goooolll ... Our t...,"[football, playing, field, goooolll, our, team...","[football, playing, field, goooolll, our, team..."


In [125]:
# Remove unnecessary columns
train_df_merged.drop(columns=["word_list_x", "word_list_y"], inplace=True)

train_df_merged.head(2)

,text,labels,dataset,language,tokens_train,token_ids,text_norm,tokens_train_norm,MT_text,words
0,"Seeking All Things Brilliant ""I want people to...",Other,CORE,English,"[▁See, king, ▁All, ▁Things, ▁Br, illian, t, ▁""...","[6872, 6048, 3164, 119175, 13008, 162076, 18, ...","seeking all things brilliant ""i want people to...","[▁seeking, ▁all, ▁things, ▁brilliant, ▁"", i, ▁...",0,"[seeking, all, things, brilliant, i, want, peo..."
1,Meet Orchid du Bois I first met Hayley Mowday ...,Other,CORE,English,"[▁Meet, ▁Or, ch, id, ▁du, ▁Bo, is, ▁I, ▁first,...","[72626, 3347, 206, 532, 115, 2460, 164, 87, 51...",meet orchid du bois i first met hayley mowday ...,"[▁meet, ▁or, ch, id, ▁du, ▁bois, ▁i, ▁first, ▁...",0,"[meet, orchid, du, bois, i, first, met, hayley..."


In [126]:
# Save the enriched dataset
train_df_merged.to_json("datasets/tokenized_datasets/X-GENRE-train-tokenized.json")

## Calculate word overlap

These are just initial experiments - to do this properly, we would need to tokenize the English translation of GINCO and of all translations of the test datasets, and then calculate the overlap in the list of words extracted from the first 512 tokens.

In [2]:
# Open the tokenized df
train_df = pd.read_json("datasets/tokenized_datasets/X-GENRE-train-tokenized.json")
train_df.head(2)

,text,labels,dataset,language,tokens_train,token_ids,text_norm,tokens_train_norm,MT_text,words
0,"Seeking All Things Brilliant ""I want people to...",Other,CORE,English,"[▁See, king, ▁All, ▁Things, ▁Br, illian, t, ▁""...","[6872, 6048, 3164, 119175, 13008, 162076, 18, ...","seeking all things brilliant ""i want people to...","[▁seeking, ▁all, ▁things, ▁brilliant, ▁"", i, ▁...",0,"[seeking, all, things, brilliant, i, want, peo..."
1,Meet Orchid du Bois I first met Hayley Mowday ...,Other,CORE,English,"[▁Meet, ▁Or, ch, id, ▁du, ▁Bo, is, ▁I, ▁first,...","[72626, 3347, 206, 532, 115, 2460, 164, 87, 51...",meet orchid du bois i first met hayley mowday ...,"[▁meet, ▁or, ch, id, ▁du, ▁bois, ▁i, ▁first, ▁...",0,"[meet, orchid, du, bois, i, first, met, hayley..."


In [9]:
# List of all train words
all_train_words = []
for text_list in train_df["words"].to_list():
	all_train_words.extend(text_list)

all_unique_train_words = list(set(all_train_words))

print(len(all_train_words))
print(len(all_unique_train_words))

527009
36548


In [3]:
# Import the final dataset with test sets
with open("manual-annotations/multilingual-genre-annotated-test-set.json") as main_file:
	main_dict = json.load(main_file)

main_dict.keys()

dict_keys(['mt', 'el', 'tr', 'sq', 'is', 'uk', 'ca', 'mk', 'hr', 'sl'])

In [4]:
df = pd.DataFrame(main_dict["sl"]["dataset"])
df.head(3)

,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids
1,CLASSLA-web.sl.1087171,Opinion/Argumentation,Kitajsko mesto duhov\nV Notranji Mongoliji ras...,The Chinese ghost city in Inner Mongolia is gr...,"CLASSLA-web.sl.1087171', 'domain': 'mladina.si'}",News,"[▁Kita, jsko, ▁mesto, ▁duhov, ▁V, ▁Notranj, i,...","[12992, 30878, 22041, 87909, 310, 240213, 14, ..."
2,CLASSLA-web.sl.1215246,Instruction,"Krompir skuhamo, olupimo in narežemo na tanke ...","Cook the potatoes, peel and cut into thin ring...","CLASSLA-web.sl.1215246', 'domain': 'emorje.com'}",Instruction,"[▁Krom, pir, ▁s, kuha, mo, ,, ▁olup, imo, ▁in,...","[128458, 21738, 91, 60863, 432, 4, 37663, 2414..."
3,CLASSLA-web.sl.1230602,Opinion/Argumentation,Danes\nJutri\nPojutrišnjem\nSo res pojedli 200...,Today tomorrow they really ate 200 kg of meats...,"CLASSLA-web.sl.1230602', 'domain': 'kr.trma.si'}",Opinion/Argumentation,"[▁Danes, ▁Ju, tri, ▁Po, ju, tri, šnje, m, ▁So,...","[85405, 3314, 3996, 663, 461, 3996, 69207, 39,..."


In [12]:
# Define an array of token overlap
token_overlap_results = {}

# Loop through the datasets and calculate English word overlap
for lang in list(main_dict.keys()):
	df = pd.DataFrame(main_dict[lang]["dataset"])

	# Add lists of English words - to do this properly, we would need to tokenize the English translation - these are just preliminary experiments
	texts = df["translation"].to_list()

	all_text_word_list = []
	all_words = []

	for text in texts:
		text = text.replace("\n", " ")

		text_list = text.split(" ")

		word_list = []

		# Take only the first 512 words
		for word in text_list[:512]:
			for punct in [".", '"', "!", "?", ":", ";", ",", "/", ")", "(", "[", "]"]:
				word = word.replace(punct, "").lower()

			if len(word) > 0:
				word_list.append(word)
				all_words.append(word)

		all_text_word_list.append(word_list)

	df["word_list"] = all_text_word_list

	print(all_words[:10])
	print("All words:")
	print(len(all_words))

	token_overlap_results = {}

	# See how many tokens do not overlap
	no_overlap_counter = 0

	print("Calculating overlap.")

	# Save tokens that overlap for further inspection
	overlap_all_words = []

	# We calculate overlap by counting how many tokens do not appear in both sets
	for token in tqdm(all_words):
		if token not in all_unique_train_words:
			no_overlap_counter += 1
		else:
			overlap_all_words.append(token)

	# Out of all tokens in test set, how many do not overlap with train set?
	no_overlap_per = no_overlap_counter/len(all_words)

	# Calculate percentage of overlap based on that
	overlap_per = 1-no_overlap_per

	print(f"Number of English words that overlap: {len(overlap_all_words)}")
	print(f"Number of different English words that overlap: {len(set(overlap_all_words))}")
	print(f"Percentage of overlap: {overlap_per}")

	# Update the dataset in the dictionary
	main_dict[lang]["dataset"] = df.to_dict()

	# Add the list of all tokens to the dictionary
	main_dict[lang]["English_word_overlap"] = {"overlap_percentage":overlap_per, "all_words": all_words, "overlap_words":overlap_all_words}

	# Add to the results
	token_overlap_results[lang] = {"percentage": overlap_per, "overlap_list_size": len(overlap_all_words), "overlap_set_size": len(set(overlap_all_words))}

['angelo', 'chetcuti', 'will', 'be', 'replacing', 'bjorn', 'vassallo', 'as', 'secretary-general', 'of']
All words:
26488
Calculating overlap.


100%|██████████| 26488/26488 [00:06<00:00, 4091.82it/s]


Number of English words that overlap: 24087
Number of different English words that overlap: 4276
Percentage of overlap: 0.9093551797040169
['update', 'pegasus', 'estiasi', 'with', 'incoming', 'calls', 'pegasus', 'estiasi', 'has', 'the']
All words:
22165
Calculating overlap.


100%|██████████| 22165/22165 [00:05<00:00, 4070.62it/s]


Number of English words that overlap: 19967
Number of different English words that overlap: 3910
Percentage of overlap: 0.900834649221746
['aöl', 'warned', 'of', 'frequent', 'negligence', 'and', 'errors', 'on', 'the', 'selection']
All words:
23379
Calculating overlap.


...

Number of English words that overlap: 17236
Number of different English words that overlap: 3574
Percentage of overlap: 0.9402651246522284
['about', 'the', 'color', 'transformer', 'product', 'for', 'smart', 'and', 'easy', 'coloring']
All words:
19381
Calculating overlap.


100%|██████████| 19381/19381 [00:04<00:00, 4148.59it/s]


Number of English words that overlap: 18112
Number of different English words that overlap: 3718
Percentage of overlap: 0.934523502399257
['the', 'chinese', 'ghost', 'city', 'in', 'inner', 'mongolia', 'is', 'growing', 'a']
All words:
19260
Calculating overlap.


100%|██████████| 19260/19260 [00:04<00:00, 4210.30it/s]

Number of English words that overlap: 18047
Number of different English words that overlap: 3677
Percentage of overlap: 0.9370197300103842


In [14]:
token_overlap_results = {}

for lang in list(main_dict.keys()):
	overlap_token_list = main_dict[lang]["English_word_overlap"]["all_words"]
	# Add to the results
	token_overlap_results[lang] = {"percentage": main_dict[lang]["English_word_overlap"]["overlap_percentage"], "overlap_list_size": len(overlap_token_list), "overlap_set_size": len(set(overlap_token_list))}

In [16]:
overlap_df = pd.DataFrame(token_overlap_results).transpose().sort_values(by="percentage", ascending=False)
overlap_df

,percentage,overlap_list_size,overlap_set_size
mk,0.940265,18331.0,4426.0
sl,0.937020,19260.0,4635.0
hr,0.934524,19381.0,4672.0
uk,0.910329,25036.0,5978.0
mt,0.909355,26488.0,6270.0
tr,0.904401,23379.0,5413.0
ca,0.903162,20085.0,4939.0
is,0.902661,23259.0,5366.0
sq,0.901896,16615.0,4382.0
el,0.900835,22165.0,5685.0


In [17]:
# Save the final dataset with test sets
with open("manual-annotations/multilingual-genre-annotated-test-set.json", "w") as main_file:
	json.dump(main_dict,main_file)

### Label level

In [18]:
# Open the tokenized train df
train_df = pd.read_json("datasets/tokenized_datasets/X-GENRE-train-tokenized.json")
train_df.head(2)

,text,labels,dataset,language,tokens_train,token_ids,text_norm,tokens_train_norm,MT_text,words
0,"Seeking All Things Brilliant ""I want people to...",Other,CORE,English,"[▁See, king, ▁All, ▁Things, ▁Br, illian, t, ▁""...","[6872, 6048, 3164, 119175, 13008, 162076, 18, ...","seeking all things brilliant ""i want people to...","[▁seeking, ▁all, ▁things, ▁brilliant, ▁"", i, ▁...",0,"[seeking, all, things, brilliant, i, want, peo..."
1,Meet Orchid du Bois I first met Hayley Mowday ...,Other,CORE,English,"[▁Meet, ▁Or, ch, id, ▁du, ▁Bo, is, ▁I, ▁first,...","[72626, 3347, 206, 532, 115, 2460, 164, 87, 51...",meet orchid du bois i first met hayley mowday ...,"[▁meet, ▁or, ch, id, ▁du, ▁bois, ▁i, ▁first, ▁...",0,"[meet, orchid, du, bois, i, first, met, hayley..."


In [19]:
# Separate the train df into label-based dfs

# Create lists of tokens that are label based
label_tokens = {}
token_count = {}
type_count = {}

for label in ['Information/Explanation', 'News', 'Instruction','Opinion/Argumentation', 'Forum', 'Prose/Lyrical', 'Legal','Promotion']:
	print(f"Processing {label}")

	label_df = train_df[train_df["labels"] == label]
	display(label_df.head(3))

	train_tokens_shortened = []

	for i in label_df["words"].to_list():
		train_tokens_shortened.extend(i)

	print(f"Number of all tokens: {len(train_tokens_shortened)}")

	# Add to dictionaries
	label_tokens[label] = train_tokens_shortened
	token_count[label] = len(train_tokens_shortened)
	type_count[label] = len(set(train_tokens_shortened))

Processing Information/Explanation


,text,labels,dataset,language,tokens_train,token_ids,text_norm,tokens_train_norm,MT_text,words
2,Abstract Objective: Reporting bias due to soci...,Information/Explanation,CORE,English,"[▁Abstract, ▁Object, ive, :, ▁Report, ing, ▁bi...","[233973, 134549, 5844, 12, 34798, 214, 333, 16...",abstract objective: reporting bias due to soci...,"[▁abstract, ▁objective, :, ▁report, ing, ▁bi, ...",0,"[abstract, objective, reporting, bias, due, to..."
3,In 2009 the song was the focus of a successful...,Information/Explanation,CORE,English,"[▁In, ▁2009, ▁the, ▁song, ▁was, ▁the, ▁focus, ...","[360, 1877, 70, 11531, 509, 70, 32153, 111, 10...",in 2009 the song was the focus of a successful...,"[▁in, ▁2009, ▁the, ▁song, ▁was, ▁the, ▁focus, ...",0,"[in, 2009, the, song, was, the, focus, of, a, ..."
39,Story: Whaling Page 4 -- M?ori and whaling Wha...,Information/Explanation,CORE,English,"[▁Story, :, ▁W, hal, ing, ▁Page, ▁4, ▁--, ▁M, ...","[30575, 12, 601, 4200, 214, 14231, 201, 4210, ...",story: whaling page 4 -- m?ori and whaling wha...,"[▁story, :, ▁w, hal, ing, ▁page, ▁4, ▁--, ▁m, ...",0,"[story, whaling, page, 4, --, mori, and, whali..."


Number of all tokens: 90496
Processing News


,text,labels,dataset,language,tokens_train,token_ids,text_norm,tokens_train_norm,MT_text,words
4,QuotW This was the week when neither rumours o...,News,CORE,English,"[▁Quo, t, W, ▁This, ▁was, ▁the, ▁week, ▁when, ...","[43851, 18, 1456, 3293, 509, 70, 5895, 3229, 2...",quotw this was the week when neither rumours o...,"[▁quo, tw, ▁this, ▁was, ▁the, ▁week, ▁when, ▁n...",0,"[quotw, this, was, the, week, when, neither, r..."
5,KaZaA claims it can't stop users sharing music...,News,CORE,English,"[▁Ka, Za, A, ▁claims, ▁it, ▁can, ', t, ▁stop, ...","[1136, 16737, 284, 140526, 442, 831, 25, 18, 7...",kazaa claims it can't stop users sharing music...,"[▁kaza, a, ▁claims, ▁it, ▁can, ', t, ▁stop, ▁u...",0,"[kazaa, claims, it, can't, stop, users, sharin..."
9,Nebraska fans checking out airfare for a trip ...,News,CORE,English,"[▁Ne, bra, ska, ▁fans, ▁checking, ▁out, ▁air, ...","[799, 2844, 937, 35992, 175199, 1810, 1831, 44...",nebraska fans checking out airfare for a trip ...,"[▁ne, bra, ska, ▁fans, ▁checking, ▁out, ▁air, ...",0,"[nebraska, fans, checking, out, airfare, for, ..."


Number of all tokens: 100620
Processing Instruction


...

Number of all tokens: 35162
Processing Legal


,text,labels,dataset,language,tokens_train,token_ids,text_norm,tokens_train_norm,MT_text,words
35,Full Terms and Conditions Eligibility to enter...,Legal,CORE,English,"[▁Full, ▁Terms, ▁and, ▁Condi, tions, ▁E, ligi,...","[9312, 165504, 136, 46347, 5256, 241, 7883, 83...",full terms and conditions eligibility to enter...,"[▁full, ▁terms, ▁and, ▁conditions, ▁e, ligi, b...",0,"[full, terms, and, conditions, eligibility, to..."
110,Commonwealth Consolidated Acts INCOME TAX ASSE...,Legal,CORE,English,"[▁Common, we, al, th, ▁Con, solid, ated, ▁Act,...","[151301, 1177, 289, 927, 1657, 97281, 27686, 2...",commonwealth consolidated acts income tax asse...,"[▁common, we, al, th, ▁consolida, ted, ▁act, s...",0,"[commonwealth, consolidated, acts, income, tax..."
122,"This Terms of Use Agreement (""Agreement"") is b...",Legal,CORE,English,"[▁This, ▁Terms, ▁of, ▁Use, ▁Agreement, ▁("", A,...","[3293, 165504, 111, 36836, 186670, 24073, 284,...","this terms of use agreement (""agreement"") is b...","[▁this, ▁terms, ▁of, ▁use, ▁agreement, ▁("", a,...",0,"[this, terms, of, use, agreement, agreement, i..."


Number of all tokens: 22944
Processing Promotion


,text,labels,dataset,language,tokens_train,token_ids,text_norm,tokens_train_norm,MT_text,words
79,Post navigation Citizen Fish are back on the r...,Promotion,CORE,English,"[▁Post, ▁navigation, ▁Citizen, ▁Fish, ▁are, ▁b...","[2795, 134470, 193223, 104796, 621, 4420, 98, ...",post navigation citizen fish are back on the r...,"[▁post, ▁navigation, ▁citizen, ▁fish, ▁are, ▁b...",0,"[post, navigation, citizen, fish, are, back, o..."
130,Win yourself a FREE copy of the BradyGames off...,Promotion,CORE,English,"[▁Win, ▁yourself, ▁a, ▁FREE, ▁copy, ▁of, ▁the,...","[17686, 31949, 10, 86697, 43658, 111, 70, 5859...",win yourself a free copy of the bradygames off...,"[▁win, ▁yourself, ▁a, ▁free, ▁copy, ▁of, ▁the,...",0,"[win, yourself, a, free, copy, of, the, bradyg..."
179,Do You Want To Know The Quick Secret for On Pa...,Promotion,CORE,English,"[▁Do, ▁You, ▁Want, ▁To, ▁Know, ▁The, ▁Quick, ▁...","[984, 2583, 42335, 717, 70829, 581, 89038, 390...",do you want to know the quick secret for on pa...,"[▁do, ▁you, ▁want, ▁to, ▁know, ▁the, ▁quick, ▁...",0,"[do, you, want, to, know, the, quick, secret, ..."


Number of all tokens: 65152


In [20]:
# Calculate numbers for each label
label_results_train = pd.DataFrame({"token_count": token_count, "type_count": type_count})

print(label_results_train.to_markdown())

|                         |   token_count |   type_count |
|:------------------------|--------------:|-------------:|
| Information/Explanation |         90496 |        13430 |
| News                    |        100620 |        14100 |
| Instruction             |         64195 |         7790 |
| Opinion/Argumentation   |         81309 |        11486 |
| Forum                   |         44728 |         7068 |
| Prose/Lyrical           |         35162 |         5509 |
| Legal                   |         22944 |         3202 |
| Promotion               |         65152 |         9694 |


### Create label-level token counts for test sets

In [21]:
# Import the final dataset with test sets
with open("manual-annotations/multilingual-genre-annotated-test-set.json") as main_file:
	main_dict = json.load(main_file)

main_dict["uk"].keys()

dict_keys(['accuracy', 'micro_f1', 'macro_f1', 'label_scores', 'dataset', 'token_overlap', 'English_word_overlap'])

In [23]:
# Do the same as with the dataset, but on every language
lang_results = {}

for lang in list(main_dict.keys()):
	print(f"Processing {lang}")

	label_token_dict = {}

	# Current df
	df = pd.DataFrame(main_dict[lang]["dataset"])
	display(df.head(2))

	if lang != "mt":
		for label in ['Information/Explanation', 'News', 'Instruction','Opinion/Argumentation', 'Forum', 'Prose/Lyrical', 'Legal','Promotion']:
			print(f"Processing {label}")

			label_df = df[df["y_true"] == label]

			# Create a list of tokens
			token_list = []

			for i in label_df["word_list"].to_list():
				token_list.extend(i)

			# Add to dictionary
			label_token_dict[label] = token_list

	elif lang == "mt":
		for label in ['Information/Explanation', 'News', 'Instruction','Opinion/Argumentation', 'Forum', 'Prose/Lyrical', 'Promotion']:
			print(f"Processing {label}")

			label_df = df[df["y_true"] == label]

			# Create a list of tokens
			token_list = []

			for i in label_df["word_list"].to_list():
				token_list.extend(i)

			# Add to dictionary
			label_token_dict[label] = token_list

	# Add to main dict
	main_dict[lang]["English_word_overlap"]["label_level_word_lists"] = label_token_dict


Processing mt


,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids,word_list
0,macocu.mt.402244,News,"Angelo Chetcuti, se jkun qed jieħu post Bjorn ...","Angelo Chetcuti, will be replacing Bjorn Vassa...",{'text_id': 'macocu.mt.402244'},News,"[▁Angel, o, ▁Che, t, cuti, ,, ▁se, ▁j, kun, ▁q...","[26902, 31, 5024, 18, 64969, 4, 40, 1647, 6262...","[angelo, chetcuti, will, be, replacing, bjorn,..."
1,macocu.mt.377203,Prose/Lyrical,Poltergeist jirreferi għal fenomeni oħra tal-m...,"Poltergeist refers to other woman's phenomena,...",{'text_id': 'macocu.mt.377203'},Opinion/Argumentation,"[▁Pol, ter, geist, ▁jir, re, feri, ▁g, ħ, al, ...","[9017, 720, 178490, 52826, 107, 26926, 706, 24...","[poltergeist, refers, to, other, woman's, phen..."


Processing Information/Explanation
Processing News
Processing Instruction
Processing Opinion/Argumentation
Processing Forum
Processing Prose/Lyrical
Processing Promotion
Processing el


,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids,word_list
0,macocu.el.1525713,Instruction,Ενημέρωση του Pegasus Estiasi με τις εισερχόμε...,Update Pegasus Estiasi with Incoming Calls\n\n...,{'text_id': 'macocu.el.1525713'},Instruction,"[▁Ενημέρωση, ▁του, ▁Pegasus, ▁Esti, asi, ▁με, ...","[236422, 385, 241060, 60271, 1544, 558, 1713, ...","[update, pegasus, estiasi, with, incoming, cal..."
1,macocu.el.3525724,Forum,Η τιμή της έκδοσης 8GB/ 128GB είναι 1.299 ευρώ...,"The price of 8GB/ 128GB is € 1,299, of the 12G...",{'text_id': 'macocu.el.3525724'},Forum,"[▁Η, ▁τιμή, ▁της, ▁έκδοση, ς, ▁8, GB, /, ▁128,...","[1700, 77118, 463, 110873, 235, 382, 8359, 64,...","[the, price, of, 8gb, 128gb, is, €, 1299, of, ..."


Processing Information/Explanation
Processing News
Processing Instruction
Processing Opinion/Argumentation
Processing Forum
Processing Prose/Lyrical
Processing Legal
Processing Promotion
Processing tr


...

Processing Information/Explanation
Processing News
Processing Instruction
Processing Opinion/Argumentation
Processing Forum
Processing Prose/Lyrical
Processing Legal
Processing Promotion
Processing hr


,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids,word_list
0,CLASSLA-web.hr.1033815,Promotion,"O proizvodu\nColor Transformer, za pametno i j...","About the Color Transformer product, for smart...","CLASSLA-web.hr.1033815', 'domain': 'hairshop.hr'}",Promotion,"[▁O, ▁proizvod, u, ▁Color, ▁Trans, former, ,, ...","[180, 43170, 34, 51193, 11062, 82772, 4, 80, 6...","[about, the, color, transformer, product, for,..."
2,CLASSLA-web.hr.1119579,Promotion,Sunčano selo / Sunny village\nNa obroncima Bil...,Sunshine / Sunny Village on the slopes of Bilo...,"CLASSLA-web.hr.1119579', 'domain': 'vikendi.com'}",Promotion,"[▁Sun, čan, o, ▁se, lo, ▁/, ▁Sunny, ▁village, ...","[7550, 17129, 31, 40, 365, 248, 151197, 54427,...","[sunshine, sunny, village, on, the, slopes, of..."


Processing Information/Explanation
Processing News
Processing Instruction
Processing Opinion/Argumentation
Processing Forum
Processing Prose/Lyrical
Processing Legal
Processing Promotion
Processing sl


,text_id,y_pred,text,translation,metadata,y_true,tokens,token_ids,word_list
1,CLASSLA-web.sl.1087171,Opinion/Argumentation,Kitajsko mesto duhov\nV Notranji Mongoliji ras...,The Chinese ghost city in Inner Mongolia is gr...,"CLASSLA-web.sl.1087171', 'domain': 'mladina.si'}",News,"[▁Kita, jsko, ▁mesto, ▁duhov, ▁V, ▁Notranj, i,...","[12992, 30878, 22041, 87909, 310, 240213, 14, ...","[the, chinese, ghost, city, in, inner, mongoli..."
2,CLASSLA-web.sl.1215246,Instruction,"Krompir skuhamo, olupimo in narežemo na tanke ...","Cook the potatoes, peel and cut into thin ring...","CLASSLA-web.sl.1215246', 'domain': 'emorje.com'}",Instruction,"[▁Krom, pir, ▁s, kuha, mo, ,, ▁olup, imo, ▁in,...","[128458, 21738, 91, 60863, 432, 4, 37663, 2414...","[cook, the, potatoes, peel, and, cut, into, th..."


Processing Information/Explanation
Processing News
Processing Instruction
Processing Opinion/Argumentation
Processing Forum
Processing Prose/Lyrical
Processing Legal
Processing Promotion


In [24]:
main_dict["sl"]["English_word_overlap"]["label_level_word_lists"]

{'Information/Explanation': ['the',
  'mansion',
  'may',
  'be',
  'the',
  'successor',
  'to',
  'one',
  'of',
  'the',
  'many',
  'sources',
  'of',
  'medieval',
  'courts',
  'in',
  'the',
  'wider',
  'vipava',
  'area',
  'it',
  'was',
  'first',
  'mentioned',
  'in',
  '1630',
  'in',
  'the',
  'census',
  'of',
  "balthazar's",
  'leg',
  'raumschüssla',
  'the',
  'lord',
  'at',
  'belnek',
  'near',
  'moravce',
  'in',
  'the',
  'gorenjska',
  'region',
  'and',
  'in',
  'vrhpolje',
  'near',
  'vipava',
  'due',
  'to',
  'its',
  'location',
  'by',
  'the',
  'water',
  'it',
  'got',
  'the',
  'german',
  'name',
  'schönau',
  'which',
  'means',
  'a',
  'beautiful',
  'log',
  'the',
  'census',
  'of',
  'his',
  'legacy',
  'shows',
  'that',
  'the',
  'ramschüssl',
  'lineage',
  'had',
  'an',
  'estate',
  'near',
  'vrhpolje',
  'in',
  '1571',
  'in',
  'the',
  'village',
  'of',
  'dolenje',
  'vipava',
  'supposedly',
  'vrhpolje',
  'like',
  '

In [26]:
token_overlap_label_results = {}
results = {}

# Loop through the datasets and labels and calculate token overlap
for lang in list(main_dict.keys()):
	print(lang)
	# Create a dict for labels
	label_overlap = {}
	label_overlap_tokens = {}

	# loop through labels
	if lang != "mt":
		for label in ['Information/Explanation', 'News', 'Instruction','Opinion/Argumentation', 'Forum', 'Prose/Lyrical', 'Legal','Promotion']:
			print(label)
			token_list_test = main_dict[lang]["English_word_overlap"]["label_level_word_lists"][label]

			token_list_train = label_tokens[label]

			# See how many tokens do not overlap
			no_overlap_counter = 0

			# Save tokens that overlap for further inspection
			overlap_token_list = []

			# We calculate overlap by counting how many tokens do not appear in both sets
			for token in tqdm(token_list_test):
				if token not in token_list_train:
					no_overlap_counter += 1
				else:
					overlap_token_list.append(token)

			# Out of all tokens in test set, how many do not overlap with train set?
			no_overlap_per = no_overlap_counter/len(token_list_test)

			# Calculate percentage of overlap based on that
			overlap_per = 1-no_overlap_per

			print(f"Percentage of overlap: {overlap_per}")

			label_overlap[label] = overlap_per
			label_overlap_tokens[label] = overlap_token_list

			# Add to the results
			results[f"{lang}-{label}"] = overlap_per

		# Add the list of all tokens to the dictionary
		main_dict[lang]["English_word_overlap"]["label_overlap_percentage"] = label_overlap
		main_dict[lang]["English_word_overlap"]["label_overlap_token_list"] = label_overlap_tokens
	else:
		# Skip "Legal" for Maltese, because we do not have this label there
		for label in ['Information/Explanation', 'News', 'Instruction','Opinion/Argumentation', 'Forum', 'Prose/Lyrical', 'Promotion']:
			print(label)
			token_list_test = main_dict[lang]["English_word_overlap"]["label_level_word_lists"][label]

			token_list_train = label_tokens[label]

			# See how many tokens do not overlap
			no_overlap_counter = 0

			# Save tokens that overlap for further inspection
			overlap_token_list = []

			# We calculate overlap by counting how many tokens do not appear in both sets
			for token in tqdm(token_list_test):
				if token not in token_list_train:
					no_overlap_counter += 1
				else:
					overlap_token_list.append(token)

			# Out of all tokens in test set, how many do not overlap with train set?
			no_overlap_per = no_overlap_counter/len(token_list_test)

			# Calculate percentage of overlap based on that
			overlap_per = 1-no_overlap_per

			print(f"Percentage of overlap: {overlap_per}")

			label_overlap[label] = overlap_per
			label_overlap_tokens[label] = overlap_token_list

			# Add to the results
			results[f"{lang}-{label}"] = overlap_per

		# Add the list of all tokens to the dictionary
		main_dict[lang]["English_word_overlap"]["label_overlap_percentage"] = label_overlap
		main_dict[lang]["English_word_overlap"]["label_overlap_token_list"] = label_overlap_tokens		

mt
Information/Explanation


100%|██████████| 6194/6194 [00:01<00:00, 6009.29it/s]


Percentage of overlap: 0.8609945108169196
News


100%|██████████| 3667/3667 [00:00<00:00, 6472.59it/s]


Percentage of overlap: 0.869375511317153
Instruction


...

Percentage of overlap: 0.8396458435809149
Legal


100%|██████████| 2644/2644 [00:00<00:00, 20064.88it/s]


Percentage of overlap: 0.8252647503782149
Promotion


100%|██████████| 2203/2203 [00:00<00:00, 8371.60it/s]

Percentage of overlap: 0.8583749432591921


In [27]:
results

{'mt-Information/Explanation': 0.8609945108169196,
 'mt-News': 0.869375511317153,
 'mt-Instruction': 0.83977303431505,
 'mt-Opinion/Argumentation': 0.8409288824383164,
 'mt-Forum': 0.7367387033398821,
 'mt-Prose/Lyrical': 0.8028455284552846,
 'mt-Promotion': 0.8367859384808537,
 'el-Information/Explanation': 0.8315346143722032,
 'el-News': 0.8443596268023749,
 'el-Instruction': 0.823250920568122,
 'el-Opinion/Argumentation': 0.8459715639810427,
 'el-Forum': 0.8088642659279779,
 'el-Prose/Lyrical': 0.8516388729154687,
 'el-Legal': 0.7653890824622532,
 'el-Promotion': 0.8212654924983692,
 'tr-Information/Explanation': 0.8175505050505051,
 'tr-News': 0.8557768924302789,
 'tr-Instruction': 0.8413502109704641,
 'tr-Opinion/Argumentation': 0.8635624237494917,
 'tr-Forum': 0.8151759220008479,
 'tr-Prose/Lyrical': 0.8123736352608169,
 'tr-Legal': 0.7867144252686421,
 'tr-Promotion': 0.8576830525953936,
 'sq-Information/Explanation': 0.8414477898782832,
 'sq-News': 0.8588074023303632,
 'sq-Inst

In [28]:
# Inspect the results
overlap_df = pd.DataFrame({"label": list(results.keys()), "overlap": list(results.values())})
overlap_df

,label,overlap
0,mt-Information/Explanation,0.860995
1,mt-News,0.869376
2,mt-Instruction,0.839773
3,mt-Opinion/Argumentation,0.840929
4,mt-Forum,0.736739
...,...,...
74,sl-Opinion/Argumentation,0.887936
75,sl-Forum,0.814320
76,sl-Prose/Lyrical,0.839646
77,sl-Legal,0.825265


In [29]:
# Save the label-level-overlap
with open("datasets/label-level-English-world-overlap.csv", "w") as file:
	overlap_df.to_csv(file)

In [ ]:
# Save the extended json dict
with open("manual-annotations/multilingual-genre-annotated-test-set.json", "w") as file:
	json.dump(main_dict, file)